In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

In [3]:
df=pd.read_csv('/content/spam-L15340.csv',engine="python",on_bad_lines='skip',encoding='latin-1')

In [ ]:
# df.columns=['spam','text']

In [4]:
df=df[['Category','Message']]

In [5]:
texts,labels=[],[]
for i,label in enumerate(df['Category']):
  texts.append(df['Message'][i])
  if label=='ham':
    labels.append(0)
  else:
    labels.append(1)
  # texts.append(df['Message'][i])
texts=np.asarray(texts)
labels=np.asarray(labels)

In [ ]:
# texts[1]

In [6]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
sequence=tokenizer.texts_to_sequences(texts)

In [ ]:
## shuffle the data

In [7]:
##padding
max_len=500
data=tf.keras.preprocessing.sequence.pad_sequences(sequence,maxlen=max_len)

In [8]:
np.random.seed(42)
indices = np.arange(df.shape[0])
np.random.shuffle(indices)

data = df.iloc[indices].reset_index(drop=True)
labels = labels[indices]

In [ ]:
## split the data into training and testing sets

In [17]:
training_sample=int(len(data)*0.8)
x_train=data[:training_sample]
y_train=labels[:training_sample]
x_test=data[training_sample:]
y_test=labels[training_sample:]


In [14]:
# Convert all columns in data to numeric, coercing non-numeric values to NaN
data = data.apply(pd.to_numeric, errors='coerce')

# Fill NaN values (optional)
data = data.fillna(0)

# Convert entire DataFrame to float32 for TensorFlow
data = data.astype('float32')

In [15]:
print(type(data), getattr(data, 'dtype', 'no dtype'))  # works for NumPy or pandas
print(type(labels), getattr(labels, 'dtype', 'no dtype'))
print(x_train.dtype if hasattr(x_train, 'dtype') else 'no dtype')
print(y_train.dtype if hasattr(y_train, 'dtype') else 'no dtype')

<class 'pandas.core.frame.DataFrame'> no dtype
<class 'numpy.ndarray'> int64
no dtype
int64


In [10]:
## Define RNN Features
max_features=5000
model=Sequential()
model.add(Embedding(max_features,32))
model.add(SimpleRNN(32))
model.add(Dense(1,activation='sigmoid'))
## compile the model
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
## Train the model

In [18]:
print(x_train.dtype)
print(y_train.dtype)

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [19]:
history=model.fit(x_train,y_train,epochs=10,batch_size=128,validation_split=0.2)

Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - acc: 0.8721 - loss: 0.4752 - val_acc: 0.8767 - val_loss: 0.3739
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8623 - loss: 0.4021 - val_acc: 0.8767 - val_loss: 0.3742
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8578 - loss: 0.4097 - val_acc: 0.8767 - val_loss: 0.3735
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8596 - loss: 0.4073 - val_acc: 0.8767 - val_loss: 0.3740
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8500 - loss: 0.4238 - val_acc: 0.8767 - val_loss: 0.3742
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8658 - loss: 0.3953 - val_acc: 0.8767 - val_loss: 0.3739
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.8616 - loss: 0.4026 - val_acc: 0.8767 - val_loss: 0.3737
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.8696 - loss: 0.3875 - val_acc: 0.8767 - val_loss: 0.3783
Epoch 9/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - acc: 0.8647 - los

In [ ]:
## evaluate the model

In [20]:
predict=model.predict(x_test)
accuracy=model.evaluate(x_test,y_test)
print('Test loss is {0:2.f} and accuracy is {1:.2f}:',accuracy[1])

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.8476 - loss: 0.4286
Test loss is {0:2.f} and accuracy is {1:.2f}: 0.8591928482055664


In [26]:
new_message="U dun say so early hor... U c already then say..."
new_message_seq=tokenizer.texts_to_sequences([new_message])
new_message_pad=tf.keras.preprocessing.sequence.pad_sequences(new_message_seq,maxlen=max_len)
prediction=model.predict(new_message_pad)
predicted_label="spam" if prediction[0][0]>0.5 else "ham"
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
[[0.28531376]]
